In [ ]:
###This script parses through the BLASTn output

In [1]:
from scipy.stats import chi2_contingency

In [5]:
###Make a list of all samples
samples = []

for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/Mapping_v4_allfc_mod.txt", "r"):
    line = line.strip()
    samples.append(line)

In [6]:
###Make a dictionary of dictionaries enumerating the number of fragments found shared between two samples
sharedElements = {}

for query in samples:
    sharedElements[query] = {}
    for subject in samples:
        if query != subject:
            sharedElements[query][subject] = 0

In [ ]:
for sample in samples:
    input_file = "/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_samples/BLASTn/filtered/filtered_" + sample + ".txt"
    for line in open(input_file, "r"):
        line = line.strip().split("\t")
        subject = line[1].split("_")[0]
        sharedElements[sample][subject] += 1
        

In [ ]:
sharedElements_path = "/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_samples/BLASTn/221004_ShareElementsCounts.txt"
sharedElements_file = open(sharedElements_path, "w")

print("Query\tSubject\tCount", file = sharedElements_file)

for query in sharedElements.keys():
    for subject, count in sharedElements[query].items():
        print(query + "\t" + subject + "\t" + str(count), file = sharedElements_file)
        
sharedElements_file.close()

In [ ]:
sharedElements_condensed = {}

for query in sharedElements.keys():
    for subject in sharedElements[query].keys():
        combo = query + "\t" + subject
        combo_alt = subject + "\t" + query
        if combo not in sharedElements_condensed.keys() and combo_alt not in sharedElements_condensed.keys():
            mean = (sharedElements[query][subject] + sharedElements[subject][query])/2
            sharedElements_condensed[combo] = mean

In [ ]:
sharedElements_condensed_path = "/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_samples/BLASTn/221004_ShareElementsCounts_condensed.txt"
sharedElements_condensed_file = open(sharedElements_condensed_path, "w")

print("Sample1\tSample2\tCount", file = sharedElements_condensed_file)

for key, value in sharedElements_condensed.items():
    print(key + "\t" + str(value), file = sharedElements_condensed_file)
    
sharedElements_condensed_file.close()

In [8]:
###Doing the same as above, but for HQ and MQ MAGs
###First, getting a list of all MAGs
MAGs = []

for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/Mapping_v9_allHQandMQ-MAGs.txt", "r"):
    line = line.strip()
    MAGs.append(line)

In [9]:
###Making a dictionary of contigs and which MAG they came from
contigMAGs = {}

for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d07_bin/dastool/HQandMQ_MAGs/MAGcontigs_map.txt", "r"):
    if line.startswith("Contig"):
        continue
    else:
        line = line.strip().split("\t")
        contig = line[0]
        MAG = line[1]
        
        contigMAGs[contig] = MAG

In [10]:
###Getting the filtered fastANI values
fastANI_MAGs = {}

for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d28_fastani_HQMQ/fastani_out_filtered.txt", "r"):
    line = line.strip().split("\t")

    MAG1 = line[0]
    MAG2 = line[1]
    score = float(line[2])
    
    key = MAG1 + "-" + MAG2
    
    fastANI_MAGs[key] = score

In [11]:
###Making a dictionary of the MAG qualities (HQ vs MQ)
HQ_MAGs = []

for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d07_bin/dastool/HQandMQ_MAGs/all_HQ_MAGs.txt", "r"):
    line = line.strip().split(".")[0]
    HQ_MAGs.append(line)

MQ_MAGs = []

for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d07_bin/dastool/HQandMQ_MAGs/all_HQandMQ_MAGs.txt", "r"):
    line = line.strip().split(".")[0]
    if line not in HQ_MAGs:
        MQ_MAGs.append(line)
        
MAG_quality = {}

for item in HQ_MAGs:
    MAG_quality[item] = "HQ"
    
for item in MQ_MAGs:
    MAG_quality[item] = "MQ"

In [12]:
###Making a dictionary of the taxanomic classification of the MAGs
MAG_taxa = {}

for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d07_bin/dastool/HQandMQ_MAGs/joint_classify.txt", "r"):
    line = line.strip()
    line = line.split("\t")
    
    MAG = line[0]
    tax = line[1]
    
    MAG_taxa[MAG] = tax

In [13]:
###Making a dictionary of the subject of origins of MAGs
MAG_subjects = {}

for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d07_bin/dastool/allbins_noUnbin/MAG_subjects.txt", "r"):
    line = line.strip().split("\t")
    MAG = line[0].split(".")[0]
    subject = line[1]
    
    if MAG in MAGs:
        MAG_subjects[MAG] = subject

In [21]:
print(f'{len(MAG_subjects)} and {len(HQ_MAGs)} and {len(MQ_MAGs)}')
#len(HQ_MAGs)
#len(MQ_MAGs)

4884 and 993 and 3891


In [ ]:
###Filtering the BLASTn results to remove duplicate MAG pairs
###This was taking forever, so ran through cluster
pairs = []

BLASTn_nodup_file = open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs_nodup.txt", "w")

for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs.txt"):
    line = line.strip()
    contig1 = line.split("\t")[0]
    contig2 = line.split("\t")[1]
    
    pair = [contig1, contig2]
    pair.sort()
    
    combo = pair[0] + "-" + pair[1]
    
    if combo not in pairs:
        pairs.append(combo)
        print(line, file = BLASTn_nodup_file)

BLASTn_nodup_file.close()

In [ ]:
###Going through the filtered (nodup) list of hits to get the MAG, subject, and ANI information
BLASTn_annot  = open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs_nodup_annot.txt", "w")

for line in open ("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs_nodup_BK.txt"):
    line = line.strip()
    contig1 = line.split("\t")[0]
    contig2 = line.split("\t")[1]
    
    MAG1 = contigMAGs[contig1]
    MAG2 = contigMAGs[contig2]
    
    subject1 = contig1.split("_")[0]
    subject2 = contig2.split("_")[0]
    
    combo = str(min(int(MAG1), int(MAG2))) + "-" + str(max(int(MAG1), int(MAG2)))
    
    if combo in fastANI_MAGs.keys():
        ANI = str(fastANI_MAGs[combo])
    else:
        ANI = "0"
        
    print(line + "\t" + MAG1 + "\t" + MAG2 + "\t" + subject1 + "\t" + subject2 + "\t" + ANI, file = BLASTn_annot)
    
BLASTn_annot.close()


In [ ]:
###Filtering through the teh annotated BLASTn hits list to filter out unneeded hits (i.e., ANI > 95%, same contig, same MAG)
BLASTn_filtered = open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs_nodup_annot_filt.txt", "w")

print("Contig1\tContig2\tpident\tlength\tmismatch\tgapopen\tqstart\tqend\tsstart\tsend\tevalue\tbitscore\tMAG1\tMAG2\tMAGqual1\tMAGqual2\tSubject1\tSubject2\tHostPair\tANI", file = BLASTn_filtered)

for line in open ("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs_nodup_BK.txt"):
    line = line.strip()
    contig1 = line.split("\t")[0]
    contig2 = line.split("\t")[1]
    
    MAG1 = contigMAGs[contig1]
    MAG2 = contigMAGs[contig2]
    
    qual1 = MAG_quality[MAG1]
    qual2 = MAG_quality[MAG2]
    
    subject1 = contig1.split("_")[0]
    subject2 = contig2.split("_")[0]
    
    host_pair = [subject1[0], subject2[0]]
    host_pair.sort()
    host_pair = host_pair[0] + "-" + host_pair[1]
    
    combo = str(min(int(MAG1), int(MAG2))) + "-" + str(max(int(MAG1), int(MAG2)))
    
    if combo in fastANI_MAGs.keys():
        ANI = str(fastANI_MAGs[combo])
    else:
        ANI = "0"
    
    spec1 = MAG_taxa[MAG1]
    spec1 = spec1.split("\t")[-1]
    spec2 = MAG_taxa[MAG2]
    spec2 = spec2.split("\t")[-1]
    
    same_spec = False
    
    if spec1 == spec2 and spec1 != "s__":
        same_spec = True
    
    
    if contig1 != contig2 and MAG1 != MAG2 and float(ANI) < 95 and same_spec == False:
        print(line + "\t" + MAG1 + "\t" + MAG2 + "\t" + qual1 + "\t" + qual2 + "\t" + subject1 + "\t" + subject2 + "\t" + host_pair + "\t" + ANI, file = BLASTn_filtered)
    
BLASTn_filtered.close()

In [ ]:
###Looking to see how many pairs of MAGs that are not the same species are possible
cow_MAGs = []
farmer_MAGs = []
office_MAGs = []

for MAG in MAGs:
    if MAG_subjects[MAG].startswith("C"):
        cow_MAGs.append(MAG)
    elif MAG_subjects[MAG].startswith("W"):
        farmer_MAGs.append(MAG)
    elif MAG_subjects[MAG].startswith("D"):
        office_MAGs.append(MAG)

In [ ]:
cow_farmer_combo = []

cow_office_combo = []

for MAG1 in cow_MAGs:
    for MAG2 in farmer_MAGs:
        cow_farmer_combo.append(MAG1 + "-" + MAG2)
        
for MAG1 in cow_MAGs:
    for MAG2 in office_MAGs:
        cow_office_combo.append(MAG1 + "-" + MAG2)

In [ ]:
counter = 0

for entry in cow_farmer_combo:
    MAG1 = entry.split("-")[0]
    MAG2 = entry.split("-")[1]
    
    combo = str(min(int(MAG1), int(MAG2))) + "-" + str(max(int(MAG1), int(MAG2)))
    
    if combo in fastANI_MAGs.keys():
        ANI = fastANI_MAGs[combo]
    else:
        ANI = 0
        
    spec1 = MAG_taxa[MAG1]
    spec1 = spec1.split("\t")[-1]
    spec2 = MAG_taxa[MAG2]
    spec2 = spec2.split("\t")[-1]
    
    same_spec = False
    
    if spec1 == spec2 and spec1 != "s__":
        same_spec = True
        
    if ANI > 95 or same_spec == True:
        counter += 1
            
counter

In [ ]:
counter = 0

for entry in cow_office_combo:
    MAG1 = entry.split("-")[0]
    MAG2 = entry.split("-")[1]
    
    combo = str(min(int(MAG1), int(MAG2))) + "-" + str(max(int(MAG1), int(MAG2)))
    
    if combo in fastANI_MAGs.keys():
        ANI = fastANI_MAGs[combo]
    else:
        ANI = 0
        
    spec1 = MAG_taxa[MAG1]
    spec1 = spec1.split("\t")[-1]
    spec2 = MAG_taxa[MAG2]
    spec2 = spec2.split("\t")[-1]
    
    same_spec = False
    
    if spec1 == spec2 and spec1 != "s__":
        same_spec = True
        
    if ANI > 95 or same_spec == True:
        counter += 1
            
counter

In [ ]:
counter = 0

for key, value in fastANI_MAGs.items():
    MAG1 = key.split("-")[0]
    MAG2 = key.split("-")[1]
    
    if MAG_subjects[MAG1].startswith("C") and MAG_subjects[MAG2].startswith("W") and value >= 95:
        counter += 1
    elif MAG_subjects[MAG1].startswith("W") and MAG_subjects[MAG2].startswith("C") and value >= 95:
        counter += 1

In [ ]:
counter = 0

for MAG in office_MAGs:
    if MAG_quality[MAG] == "HQ":
        counter += 1
        
counter

In [ ]:
###Going through the list of the identified BLAST hits to see which ones include ARGs
###First, creating a dictionaries of the ARG coordinates of the contigs
contig_ARGcoordinates = {}
contig_ARGannot = {}
contig_ARGORFID = {}
contig_abxClasses = {}

for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d12_shortbred_HQMQ/230118_shortBRED_ORFID_contigs_v2.txt", "r"):
    if line.startswith("MAG"):
        continue
    else:
        line = line.strip().split("\t")
        ORFID = line[1]
        abxClass = line[2]
        annot = line[3]
        contig = line[4]
        start = line[5]
        stop = line[6]
        
        if contig in contig_ARGcoordinates.keys():
            contig_ARGcoordinates[contig].append(start + "\t" + stop)
        else:
            contig_ARGcoordinates[contig] = []
            contig_ARGcoordinates[contig].append(start + "\t" + stop)
            
        if contig in contig_ARGannot.keys():
            contig_ARGannot[contig].append(annot)
        else:
            contig_ARGannot[contig] = []
            contig_ARGannot[contig].append(annot)
            
        if contig in contig_abxClasses.keys():
            contig_abxClasses[contig].append(abxClass)
        else:
            contig_abxClasses[contig] = []
            contig_abxClasses[contig].append(abxClass)
            
        if contig in contig_ARGORFID.keys():
            contig_ARGORFID[contig].append(ORFID)
        else:
            contig_ARGORFID[contig] = []
            contig_ARGORFID[contig].append(ORFID)

In [ ]:
CW_hits = []

for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs_nodup_annot_filt.txt", "r"):
    if line.startswith("Contig1"):
        continue
    else:
        line = line.strip()
        if "C-W" in line:
            CW_hits.append(line)

In [ ]:
CW_hits_ARG = []

for line in CW_hits:
    contig1 = line.split("\t")[0]
    contig2 = line.split("\t")[1]
    
    start1 = int(line.split("\t")[6])
    stop1 = int(line.split("\t")[7])
    
    if start1 > stop1:
        temp = start1
        start1 = stop1
        stop1 = temp
        
    start2 = int(line.split("\t")[8])
    stop2 = int(line.split("\t")[9])
    
    if start2 > stop2:
        temp = start2
        start2 = stop2
        stop2 = temp
    
    ARGs = ""
    
    if contig1 not in contig_ARGcoordinates.keys() and contig2 not in contig_ARGcoordinates.keys():
        CW_hits_ARG.append(line + "\tNone")
    
    else:
        
        if contig1 in contig_ARGcoordinates.keys():
            for i in range(len(contig_ARGcoordinates[contig1])):
                start = int(contig_ARGcoordinates[contig1][i].split("\t")[0])
                stop = int(contig_ARGcoordinates[contig1][i].split("\t")[1])
                cov = 0

                if start > stop:
                    temp = start
                    start = stop
                    stop = temp

                if start > start1 and start < stop1 and stop > start1 and stop < stop1:
                    cov = 100
                elif start > start1 and start < stop1 and stop > start1 and stop > stop1:
                    cov = (stop1 - start)/(stop - start)
                    cov = round(cov * 100)
                elif start < start1 and start < stop1 and stop > start1 and stop < stop1:
                    cov = (stop - start1)/(stop - start)
                    cov = round(cov * 100)
                elif start < start1 and start < stop1 and stop > start1 and stop > stop1:
                    cov = (stop1 - start1)/(stop - start)
                    cov = round(cov * 100)
                else:
                    cov = 0
                    
                if cov == 100:
                    if ARGs == "":
                        ARGs = contig1 + "," + contig_abxClasses[contig1][i] + "," + contig_ARGannot[contig1][i] + "," + str(cov)
                    else:
                        ARGs = ARGs + "\t" + contig1 + "," + contig_abxClasses[contig1][i] + "," + contig_ARGannot[contig1][i] + "," + str(cov)
                        
        if contig2 in contig_ARGcoordinates.keys():
            for i in range(len(contig_ARGcoordinates[contig2])):
                start = int(contig_ARGcoordinates[contig2][i].split("\t")[0])
                stop = int(contig_ARGcoordinates[contig2][i].split("\t")[1])
                cov = 0

                if start > stop:
                    temp = start
                    start = stop
                    stop = temp

                if start > start2 and start < stop2 and stop > start2 and stop < stop2:
                    cov = 100
                elif start > start2 and start < stop2 and stop > start2 and stop > stop2:
                    cov = (stop2 - start)/(stop - start)
                    cov = round(cov * 100)
                elif start < start2 and start < stop2 and stop > start2 and stop < stop2:
                    cov = (stop - start2)/(stop - start)
                    cov = round(cov * 100)
                elif start < start2 and start < stop2 and stop > start2 and stop > stop2:
                    cov = (stop2 - start2)/(stop - start)
                    cov = round(cov * 100)
                else:
                    cov = 0
                    
                if cov == 100:
                    if ARGs == "":
                        ARGs = contig2 + "," + contig_abxClasses[contig2][i] + "," + contig_ARGannot[contig2][i] + "," + str(cov)
                    else:
                        ARGs = ARGs + "\t" + contig2 + "," + contig_abxClasses[contig2][i] + "," + contig_ARGannot[contig2][i] + "," + str(cov)
                        
        if ARGs == "":
            CW_hits_ARG.append(line + "\tNone")
        else:
            CW_hits_ARG.append(line + "\t" + ARGs)

In [ ]:
CW_hits_ARG

In [ ]:
counter = 0

for line in CW_hits_ARG:
    if line.split("\t")[-1] != "None":
        counter += 1

counter

In [ ]:
CW_hits_ARG_file = open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs_nodup_annot_filt_CW_ARGs_100cov.txt", "w")

print("Contig1\tContig2\tpident\tlength\tmismatch\tgapopen\tqstart\tqend\tsstart\tsend\tevalue\tbitscore\tMAG1\tMAG2\tMAGqual1\tMAGqual2\tSubject1\tSubject2\tHostPair\tANI\tARGs", file = CW_hits_ARG_file)

for line in CW_hits_ARG:
    print(line.strip(), file = CW_hits_ARG_file)

CW_hits_ARG_file.close()

In [ ]:
###For many shared fragments, matching ARGs were identified from both the query and the subject, which is great and expected
###Now going thorugh the entries to keep only unique ARG-matched ORFs for each BLAST hit
CW_ARGs_unique = []

for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs_nodup_annot_filt_CW_ARGs_100cov.txt", "r"):
    if line.startswith("Contig1"):
        continue
    else:
        line = line.strip().split("\t")
        
        contig1 = line[0]
        contig2 = line[1]
        
        pident = line[2]
        length = line[3]
        
        MAG1 = line[12]
        MAG2 = line[13]
        
        taxa1 = MAG_taxa[MAG1]
        taxa1 = taxa1.split(";")
        domain1 = taxa1[0]
        phylum1 = taxa1[1]
        class1 = taxa1[2]
        order1 = taxa1[3]
        family1 = taxa1[4]
        genus1 = taxa1[5]
        species1 = taxa1[6]
        
        taxa2 = MAG_taxa[MAG2]
        taxa2 = taxa2.split(";")
        domain2 = taxa2[0]
        phylum2 = taxa2[1]
        class2 = taxa2[2]
        order2 = taxa2[3]
        family2 = taxa2[4]
        genus2 = taxa2[5]
        species2 = taxa2[6]
        
        if family1 == family2:
            families = "Same family"
        else:
            families = "Different families"
            
        if genus1 == genus2:
            genera = "Same genera"
        else:
            genera = "Different genera"
        
        subject1 = line[16]
        subject2 = line[17]
        
        ARGs = line[20:]
        
        if line[-1] == "None":
            CW_ARGs_unique.append("\t".join([contig1, contig2, pident, length, MAG1, MAG2, domain1, phylum1, class1, order1, family1, genus1, species1, domain2, phylum2, class2, order2, family2, genus2, species2, families, genera, subject1, subject2, line[-1]]))
        else:
            unique_ORFs = []
            unique_entries = []
            
            for item in ARGs:
                item = item.split(",")
                ARGclass = item[1]
                ORF = item[2]
                if ORF not in unique_ORFs:
                    unique_entries.append(ARGclass + "," + ORF)
                    unique_ORFs.append(ORF)
            
            unique_entries = "\t".join(unique_entries)
            CW_ARGs_unique.append("\t".join([contig1, contig2, pident, length, MAG1, MAG2, domain1, phylum1, class1, order1, family1, genus1, species1, domain2, phylum2, class2, order2, family2, genus2, species2, families, genera, subject1, subject2, unique_entries]))

In [ ]:
CW_ARGs_unique_file = open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs_nodup_annot_filt_CW_ARGs_100cov_uniqueORFs.txt", "w")
CW_onlyARGs_unique_file = open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs_nodup_annot_filt_CW_onlyARGs_100cov_uniqueORFs.txt", "w")

print("Contig1\tContig2\tpident\tlength\tMAG1\tMAG2\tDomain1\tPhylum1\tClass1\tOrder1\tFamily1\tGenus1\tSpecies1\tDomain2\tPhylum2\tClass2\tOrder2\tFamily2\tGenus2\tSpecies2\tFamilies\tGenera\tSubject1\tSubject2\tARGs", file = CW_ARGs_unique_file)
print("Contig1\tContig2\tpident\tlength\tMAG1\tMAG2\tDomain1\tPhylum1\tClass1\tOrder1\tFamily1\tGenus1\tSpecies1\tDomain2\tPhylum2\tClass2\tOrder2\tFamily2\tGenus2\tSpecies2\tFamilies\tGenera\tSubject1\tSubject2\tARGs", file = CW_onlyARGs_unique_file)

for line in CW_ARGs_unique:
    print(line, file = CW_ARGs_unique_file)
    if line.split("\t")[-1] != "None":
        print(line, file = CW_onlyARGs_unique_file)
        
CW_ARGs_unique_file.close()
CW_onlyARGs_unique_file.close()

In [ ]:
###Looking at classes of abx which are represented by shared ARGs
sharedAbxClasses = {}

for line in CW_ARGs_unique:
    if line.split("\t")[-1] != "None":
        ARGs = line.split("\t")[24:]
        for item in ARGs:
            ARGclass = item.split(",")[0]
            if ARGclass in sharedAbxClasses.keys():
                sharedAbxClasses[ARGclass] += 1
            else:
                sharedAbxClasses[ARGclass] = 1

In [ ]:
sum(sharedAbxClasses.values())

In [ ]:
###Looking at ORFs that are encoded within the sared fragments to do GO-term analysis
######First, creating a dictionary (MAGs) of dictionaries (contigs) where the info about encoded ORFs or releavant contigs is storred
contig_ORFs = {}

for line in CW_ARGs_unique:
    line = line.split("\t")
    MAG1 = line[4]
    MAG2 = line[5]
    
    contig1 = line[0]
    contig2 = line[1]
    
    if MAG1 not in contig_ORFs.keys():
        contig_ORFs[MAG1] = {}
        
    if MAG2 not in contig_ORFs.keys():
        contig_ORFs[MAG2] = {}
        
    if contig1 not in contig_ORFs[MAG1].keys():
        contig_ORFs[MAG1][contig1] = []
        
    if contig2 not in contig_ORFs[MAG2].keys():
        contig_ORFs[MAG2][contig2] = []

In [ ]:
######Bakta renames contigs, so generating a dictionary to link the new IDs to the old contig IDs
contig_names = {}

for MAG in contig_ORFs.keys():
    contig_names[MAG] = {}

    for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d25_bakta_HQMQ/" + MAG + "/" + MAG + ".log", "r"):
        if "INFO - UTILS - qc: revised sequence:" in line:
            line = line.strip().split(" ")
            newID = line[9][:-1].split("=")[1]
            oldID = line[10][:-1].split("=")[1]
            contig_names[MAG][oldID] = newID

In [ ]:
######Now, getting the ORFs that are encoded in each of the relevant contigs
for MAG in contig_ORFs.keys():
    for contig in contig_ORFs[MAG].keys():
        new_contig_name = contig_names[MAG][contig]
        for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d25_bakta_HQMQ/" + MAG + "/" + MAG + ".gff3", "r"):
            if line.startswith(new_contig_name) and "region" not in line:
                line = line.strip().split("\t")
                ORFtype = line[2]
                start = line[3]
                stop = line[4]
                strand = line[6]
                
                ID = line[8].split(";")[0].split("=")[1]
                name = line[8].split(";")[1].split("=")[1]
                
                entry = ORFtype + "\t" + start + "\t" + stop + "\t" + strand + "\t" + ID + "\t" + name
                
                contig_ORFs[MAG][contig].append(entry)

In [ ]:
######Now going through each of the hits to see which ORFs are found within
CW_hits_ORF = []

for line in CW_hits:
    contig1 = line.split("\t")[0]
    contig2 = line.split("\t")[1]
    
    start1 = int(line.split("\t")[6])
    stop1 = int(line.split("\t")[7])
    
    if start1 > stop1:
        temp = start1
        start1 = stop1
        stop1 = temp
        
    start2 = int(line.split("\t")[8])
    stop2 = int(line.split("\t")[9])
    
    if start2 > stop2:
        temp = start2
        start2 = stop2
        stop2 = temp
    
    MAG1 = line.split("\t")[12]
    MAG2 = line.split("\t")[13]
    
    ORFs = ""
    
    for i in range(len(contig_ORFs[MAG1][contig1])):
        ORFtype = contig_ORFs[MAG1][contig1][i].split("\t")[0]
        start = int(contig_ORFs[MAG1][contig1][i].split("\t")[1])
        stop = int(contig_ORFs[MAG1][contig1][i].split("\t")[2])
        ID = contig_ORFs[MAG1][contig1][i].split("\t")[4]
        name = contig_ORFs[MAG1][contig1][i].split("\t")[5]
        cov = 0

        if start > stop:
            temp = start
            start = stop
            stop = temp

        if start > start1 and start < stop1 and stop > start1 and stop < stop1:
            cov = 100

        if cov == 100:
            if ORFs == "":
                ORFs = contig1 + "," + ORFtype + "," + ID + "," + name
            else:
                ORFs = ORFs + "\t" + contig1 + "," + ORFtype + "," + ID + "," + name
                        
    for i in range(len(contig_ORFs[MAG2][contig2])):
        ORFtype = contig_ORFs[MAG2][contig2][i].split("\t")[0]
        start = int(contig_ORFs[MAG2][contig2][i].split("\t")[1])
        stop = int(contig_ORFs[MAG2][contig2][i].split("\t")[2])
        ID = contig_ORFs[MAG2][contig2][i].split("\t")[4]
        name = contig_ORFs[MAG2][contig2][i].split("\t")[5]
        cov = 0

        if start > stop:
            temp = start
            start = stop
            stop = temp

        if start > start2 and start < stop2 and stop > start2 and stop < stop2:
            cov = 100

        if cov == 100:
            if ORFs == "":
                ORFs = contig2 + "," + ORFtype + "," + ID + "," + name
            else:
                ORFs = ORFs + "\t" + contig2 + "," + ORFtype + "," + ID + "," + name
                        
    if ORFs == "":
        CW_hits_ORF.append(line + "\tNone")
    else:
        CW_hits_ORF.append(line + "\t" + ORFs)

In [ ]:
CW_hits_ORF_file = open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs_nodup_annot_filt_CW_ORFs_100cov.txt", "w")

print("Contig1\tContig2\tpident\tlength\tmismatch\tgapopen\tqstart\tqend\tsstart\tsend\tevalue\tbitscore\tMAG1\tMAG2\tMAGqual1\tMAGqual2\tSubject1\tSubject2\tHostPair\tANI\tORFs", file = CW_hits_ORF_file)

for line in CW_hits_ORF:
    print(line.strip(), file = CW_hits_ORF_file)

CW_hits_ORF_file.close()

In [ ]:
######Getting only farmer ORFs
CW_hits_farmerORF = []

for line in CW_hits_ORF:
    line = line.split("\t")
    entry = line[0:20]
    ORFs = []
    for ORF in line[20:]:
        if ORF.startswith("W"):
            ORFs.append(ORF)
    if len(ORFs) == 0:
        ORFs.append("None")
    entry = entry + ORFs
    CW_hits_farmerORF.append("\t".join(entry))

In [ ]:
CW_hits_farmerORF_file = open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs_nodup_annot_filt_CW_farmerORFs_100cov.txt", "w")

print("Contig1\tContig2\tpident\tlength\tmismatch\tgapopen\tqstart\tqend\tsstart\tsend\tevalue\tbitscore\tMAG1\tMAG2\tMAGqual1\tMAGqual2\tSubject1\tSubject2\tHostPair\tANI\tORFs", file = CW_hits_farmerORF_file)

for line in CW_hits_farmerORF:
    print(line, file = CW_hits_farmerORF_file)
    
CW_hits_farmerORF_file.close()

In [ ]:
######Getting the DNA and AA sequences for the farmer ORFs
DNA_seqs = open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/GOterm/230202_allFarmORFs_DNAseq.ffn", "w")
AA_seqs = open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/GOterm/230202_allFarmORFs_AAseq.faa", "w")

unique_ORFs = []

for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs_nodup_annot_filt_CW_farmerORFs_100cov.txt", "r"):
    if line.startswith("Contig1"):
        continue
    else:
        line = line.strip().split("\t")
        
        if line[0].startswith("W"):
            MAG = line[12]
        else:
            MAG = line[13]
        
        if line[20] != "None":
            for entry in line[20:]:
                if entry not in unique_ORFs:
                    unique_ORFs.append(entry)
                    ORFID = entry.split(",")[2]
                    path = "/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d25_bakta_HQMQ/" + MAG + "/" + MAG
                    
                    seqs = open(path + ".faa", "r")
                    seqs = seqs.readlines()
                    for i in range(0,len(seqs),2):
                        if ORFID in seqs[i]:
                            print(seqs[i].strip(), file = AA_seqs)
                            print(seqs[i+1].strip(), file = AA_seqs)
                            
                    
                    seqs = open(path + ".ffn", "r")
                    seqs = seqs.readlines()
                    for i in range(0,len(seqs),2):
                        if ORFID in seqs[i]:
                            print(seqs[i].strip(), file = DNA_seqs)
                            print(seqs[i+1].strip(), file = DNA_seqs)
                            
DNA_seqs.close()
AA_seqs.close()

In [ ]:
len(unique_ORFs)

In [ ]:
###Setting up the files for chord diagrams
CW_hits_chord = []

CW_hits_families = {}

for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs_nodup_annot_filt_CW_ARGs_100cov_uniqueORFs.txt", "r"):
    if line.startswith("Contig"):
        continue
    else:
        line = line.strip().split("\t")
        contig1 = line[0]
        contig2 = line[1]
        
        family1 = line[10]
        family2 = line[17]
        
        if family1 not in CW_hits_families:
            CW_hits_families[family1] = 1
        else:
            CW_hits_families[family1] += 1
            
        if family2 not in CW_hits_families:
            CW_hits_families[family2] = 1
        else:
            CW_hits_families[family2] += 1
        
        if line[24] == "None":
            ARGs = line[24]
        else:
            ARGs = "Present"
        
        families = line[20]
        
        if contig1.startswith("W"):
            family_W = contig1[0] + "-" + family1
            family_C = contig2[0] + "-" + family2
        else:
            family_W = contig2[0] + family2
            family_C = contig1[0] + family1
            
        CW_hits_chord.append("\t".join([family_W, family_C, family_W[2:], family_C[2:], families, ARGs]))
        
CW_hits_families = sorted(CW_hits_families.items(), key=lambda x:x[1], reverse=True)
CW_hits_families = dict(CW_hits_families)

In [ ]:
######Getting the top 10 families represented in the identified hits
CW_hits_top10families = list(dict(sorted(CW_hits_families.items(), key=lambda x:x[1], reverse=True)[:10]).keys())

In [ ]:
CW_hits_top10families

In [ ]:
CW_hits_chord_top10fam = []

CW_hits_chord_top10fam.append("W_family_alt\tC_family_alt\tCount\tW_family\tC_family\tFamilies\tARG")

for line in CW_hits_chord:
    line = line.split("\t")
    family_W = line[2]
    family_C = line[3]
    families = line[4]
    ARGs = line[5]
    
    if family_W in CW_hits_top10families:
        family_W = "W-" + family_W
    else:
        family_W = "W-Other"
        
    if family_C in CW_hits_top10families:
        family_C = "C-" + family_C
    else:
        family_C = "C-Other"
        
    CW_hits_chord_top10fam.append("\t".join([family_W, family_C, "1", family_W[2:], family_C[2:], families, ARGs]))

In [ ]:
CW_hits_chord_top10fam_file = open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/ChordDiagram/CW_hitsBLASTn_top10families_ARG.txt", "w")

for line in CW_hits_chord_top10fam:
    print(line, file = CW_hits_chord_top10fam_file)
    
CW_hits_chord_top10fam_file.close()

In [ ]:
###Setting up the file for the chord diagram of ARG-only hits
ARGs_top10 = [
    "TETRACYCLINE", "TRIMETHOPRIM", "GLYCOPEPTIDE", "BETA.LACTAM", "LINCOSAMIDE",
    "MACROLIDE", "PHENICOL", "QUINOLONE", "AMINOGLYCOSIDE", "FOSFOMYCIN"
             ]

In [ ]:
ARGs_top10

In [ ]:
CW_hits_ARGonly_top10fam_chord = []

CW_hits_ARGonly_top10fam_chord.append("W_family_alt\tC_family_alt\tCount\tW_family\tC_family\tFamilies\tARG")

for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs_nodup_annot_filt_CW_onlyARGs_100cov_uniqueORFs.txt", "r"):
    if line.startswith("Contig"):
        continue
    else:
        line = line.strip().split("\t")
        contig1 = line[0]
        contig2 = line[1]
        
        family1 = line[10]
        family2 = line[17]
        
        if family1 not in CW_hits_top10families:
            family1 = "Other"
            
        if family2 not in CW_hits_top10families:
            family2 = "Other"
        
        if contig1.startswith("W"):
            family_W = "W-" + family1
            family_C = "C-" + family2
        else:
            family_W = "W-" + family2
            family_C = "C-" + family1
            
        families = line[20]
        
        ARGs = line[24:]
        
        for ARG in ARGs:
            ARG = ARG.split(",")[0]
            if ARG not in ARGs_top10:
                ARG = "Other"
            CW_hits_ARGonly_top10fam_chord.append("\t".join([family_W, family_C, "1", family_W[2:], family_C[2:], families, ARG]))

In [ ]:
CW_hits_ARGonly_top10fam_chord_file = open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/ChordDiagram/CW_hitsBLASTn_top10families_onlyARG.txt", "w")

for line in CW_hits_ARGonly_top10fam_chord:
    print(line, file = CW_hits_ARGonly_top10fam_chord_file)
    
CW_hits_ARGonly_top10fam_chord_file.close()

In [ ]:
ARGs_top3 = [
    "TETRACYCLINE", "LINCOSAMIDE", "MACROLIDE"
]

In [ ]:
CW_hits_top3ARGonly_top10fam_chord = []

CW_hits_top3ARGonly_top10fam_chord.append("W_family_alt\tC_family_alt\tCount\tW_family\tC_family\tFamilies\tARG")

for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs_nodup_annot_filt_CW_onlyARGs_100cov_uniqueORFs.txt", "r"):
    if line.startswith("Contig"):
        continue
    else:
        line = line.strip().split("\t")
        contig1 = line[0]
        contig2 = line[1]
        
        family1 = line[10]
        family2 = line[17]
        
        if family1 not in CW_hits_top10families:
            family1 = "Other"
            
        if family2 not in CW_hits_top10families:
            family2 = "Other"
        
        if contig1.startswith("W"):
            family_W = "W-" + family1
            family_C = "C-" + family2
        else:
            family_W = "W-" + family2
            family_C = "C-" + family1
            
        families = line[20]
        
        ARGs = line[24:]
        
        for ARG in ARGs:
            ARG = ARG.split(",")[0]
            if ARG not in ARGs_top3:
                ARG = "Other"
            CW_hits_top3ARGonly_top10fam_chord.append("\t".join([family_W, family_C, "1", family_W[2:], family_C[2:], families, ARG]))

In [ ]:
CW_hits_top3ARGonly_top10fam_chord_file = open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/ChordDiagram/CW_hitsBLASTn_top10families_onlyARGtop3.txt", "w")

for line in CW_hits_top3ARGonly_top10fam_chord:
    print(line, file = CW_hits_top3ARGonly_top10fam_chord_file)
    
CW_hits_top3ARGonly_top10fam_chord_file.close()

In [ ]:
###Looking at the exact genes that were found in the shared fragments
######First, creating a dictionary of the gene assignments for all ARG ORFIDs
ARG_ORFs = {}

In [ ]:
###Looking at the exact genes that were found in the shared fragments
######First, creating a dictionary of the gene assignments for all ARG ORFIDs
ARG_ORFs = {}

for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d12_shortbred_HQMQ/221230_shortBREDjoint_ORFID_v3.txt", "r", errors="ignore"):
    if not line.startswith("MAG"):
        line = line.strip().split("\t")
        ARG = line[1]
        ORFID = line[2].split(" ")[0]
        if len(ORFID) != 12:
            ORFID = ORFID[1:]
        ARG_ORFs[ORFID] = ARG

In [ ]:
######Now (as above), parsing through the BLASTn to get the encoded ARGs
CW_hits_ARG = []

for line in CW_hits:
    contig1 = line.split("\t")[0]
    contig2 = line.split("\t")[1]
    
    start1 = int(line.split("\t")[6])
    stop1 = int(line.split("\t")[7])
    
    if start1 > stop1:
        temp = start1
        start1 = stop1
        stop1 = temp
        
    start2 = int(line.split("\t")[8])
    stop2 = int(line.split("\t")[9])
    
    if start2 > stop2:
        temp = start2
        start2 = stop2
        stop2 = temp
    
    ARGs = ""
    
    if contig1 not in contig_ARGcoordinates.keys() and contig2 not in contig_ARGcoordinates.keys():
        CW_hits_ARG.append(line + "\tNone")
    
    else:
        
        if contig1 in contig_ARGcoordinates.keys():
            for i in range(len(contig_ARGcoordinates[contig1])):
                start = int(contig_ARGcoordinates[contig1][i].split("\t")[0])
                stop = int(contig_ARGcoordinates[contig1][i].split("\t")[1])
                cov = 0

                if start > stop:
                    temp = start
                    start = stop
                    stop = temp

                if start > start1 and start < stop1 and stop > start1 and stop < stop1:
                    cov = 100
                elif start > start1 and start < stop1 and stop > start1 and stop > stop1:
                    cov = (stop1 - start)/(stop - start)
                    cov = round(cov * 100)
                elif start < start1 and start < stop1 and stop > start1 and stop < stop1:
                    cov = (stop - start1)/(stop - start)
                    cov = round(cov * 100)
                elif start < start1 and start < stop1 and stop > start1 and stop > stop1:
                    cov = (stop1 - start1)/(stop - start)
                    cov = round(cov * 100)
                else:
                    cov = 0
                    
                if cov == 100:
                    if ARGs == "":
                        ARGs = contig1 + "," + ARG_ORFs[contig_ARGORFID[contig1][i]] + "," + contig_abxClasses[contig1][i] + "," + contig_ARGannot[contig1][i] + "," + str(cov)
                    else:
                        ARGs = ARGs + "\t" + contig1 + "," + ARG_ORFs[contig_ARGORFID[contig1][i]] + "," + contig_abxClasses[contig1][i] + "," + contig_ARGannot[contig1][i] + "," + str(cov)
                        
        if contig2 in contig_ARGcoordinates.keys():
            for i in range(len(contig_ARGcoordinates[contig2])):
                start = int(contig_ARGcoordinates[contig2][i].split("\t")[0])
                stop = int(contig_ARGcoordinates[contig2][i].split("\t")[1])
                cov = 0

                if start > stop:
                    temp = start
                    start = stop
                    stop = temp

                if start > start2 and start < stop2 and stop > start2 and stop < stop2:
                    cov = 100
                elif start > start2 and start < stop2 and stop > start2 and stop > stop2:
                    cov = (stop2 - start)/(stop - start)
                    cov = round(cov * 100)
                elif start < start2 and start < stop2 and stop > start2 and stop < stop2:
                    cov = (stop - start2)/(stop - start)
                    cov = round(cov * 100)
                elif start < start2 and start < stop2 and stop > start2 and stop > stop2:
                    cov = (stop2 - start2)/(stop - start)
                    cov = round(cov * 100)
                else:
                    cov = 0
                    
                if cov == 100:
                    if ARGs == "":
                        ARGs = contig2 + "," + ARG_ORFs[contig_ARGORFID[contig2][i]] + "," + contig_abxClasses[contig2][i] + "," + contig_ARGannot[contig2][i] + "," + str(cov)
                    else:
                        ARGs = ARGs + "\t" + contig2 + "," + ARG_ORFs[contig_ARGORFID[contig2][i]] + "," + contig_abxClasses[contig2][i] + "," + contig_ARGannot[contig2][i] + "," + str(cov)
                        
        if ARGs == "":
            CW_hits_ARG.append(line + "\tNone")
        else:
            CW_hits_ARG.append(line + "\t" + ARGs)

In [ ]:
CW_hits_ARG_file = open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs_nodup_annot_filt_CW_ARGs_ShortbredIDs_100cov.txt", "w")

print("Contig1\tContig2\tpident\tlength\tmismatch\tgapopen\tqstart\tqend\tsstart\tsend\tevalue\tbitscore\tMAG1\tMAG2\tMAGqual1\tMAGqual2\tSubject1\tSubject2\tHostPair\tANI\tARGs", file = CW_hits_ARG_file)

for line in CW_hits_ARG:
    print(line.strip(), file = CW_hits_ARG_file)

CW_hits_ARG_file.close()

In [ ]:
###Checking to see if the ARGs have the same annotations across different samples
annot_ARGs = {}

for line in CW_hits_ARG:
    line = line.split("\t")
    if line[-1] != "None":
        ARGs = line[20:]
        for hit in ARGs:
            hit = hit.split(",")
            ARG = hit[1]
            annot = hit[3]
            if ARG in annot_ARGs.keys() and annot not in annot_ARGs[ARG]:
                annot_ARGs[ARG].append(annot)
            else:
                annot_ARGs[ARG] = [annot]

In [ ]:
list(annot_ARGs.keys())

In [ ]:
###For many shared fragments, matching ARGs were identified from both the query and the subject, which is great and expected
###Now going thorugh the entries to keep only unique ARG-matched ORFs for each BLAST hit
CW_ARGs_unique = []

for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs_nodup_annot_filt_CW_ARGs_ShortbredIDs_100cov.txt", "r"):
    if line.startswith("Contig1"):
        continue
    else:
        line = line.strip().split("\t")
        
        contig1 = line[0]
        contig2 = line[1]
        
        pident = line[2]
        length = line[3]
        
        MAG1 = line[12]
        MAG2 = line[13]
        
        taxa1 = MAG_taxa[MAG1]
        taxa1 = taxa1.split(";")
        domain1 = taxa1[0]
        phylum1 = taxa1[1]
        class1 = taxa1[2]
        order1 = taxa1[3]
        family1 = taxa1[4]
        genus1 = taxa1[5]
        species1 = taxa1[6]
        
        taxa2 = MAG_taxa[MAG2]
        taxa2 = taxa2.split(";")
        domain2 = taxa2[0]
        phylum2 = taxa2[1]
        class2 = taxa2[2]
        order2 = taxa2[3]
        family2 = taxa2[4]
        genus2 = taxa2[5]
        species2 = taxa2[6]
        
        if family1 == family2:
            families = "Same family"
        else:
            families = "Different families"
            
        if genus1 == genus2:
            genera = "Same genera"
        else:
            genera = "Different genera"
        
        subject1 = line[16]
        subject2 = line[17]
        
        ARGs = line[20:]
        
        if line[-1] == "None":
            CW_ARGs_unique.append("\t".join([contig1, contig2, pident, length, MAG1, MAG2, domain1, phylum1, class1, order1, family1, genus1, species1, domain2, phylum2, class2, order2, family2, genus2, species2, families, genera, subject1, subject2, line[-1]]))
        else:
            unique_ORFs = []
            unique_entries = []
            
            for item in ARGs:
                item = item.split(",")
                ARG = item[1]
                ARGclass = item[2]
                ORF = item[3]
                if ORF not in unique_ORFs:
                    unique_entries.append(f"{ARG}, {ARGclass}, {ORF}")
                    unique_ORFs.append(ORF)
            
            unique_entries = "\t".join(unique_entries)
            CW_ARGs_unique.append("\t".join([contig1, contig2, pident, length, MAG1, MAG2, domain1, phylum1, class1, order1, family1, genus1, species1, domain2, phylum2, class2, order2, family2, genus2, species2, families, genera, subject1, subject2, unique_entries]))

In [ ]:
CW_ARGs_unique_file = open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs_nodup_annot_filt_CW_ARGs_ShortbredIDs_100cov_uniqueORFs.txt", "w")
CW_onlyARGs_unique_file = open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs_nodup_annot_filt_CW_onlyARGs_ShortbredIDs_100cov_uniqueORFs.txt", "w")

print("Contig1\tContig2\tpident\tlength\tMAG1\tMAG2\tDomain1\tPhylum1\tClass1\tOrder1\tFamily1\tGenus1\tSpecies1\tDomain2\tPhylum2\tClass2\tOrder2\tFamily2\tGenus2\tSpecies2\tFamilies\tGenera\tSubject1\tSubject2\tARGs", file = CW_ARGs_unique_file)
print("Contig1\tContig2\tpident\tlength\tMAG1\tMAG2\tDomain1\tPhylum1\tClass1\tOrder1\tFamily1\tGenus1\tSpecies1\tDomain2\tPhylum2\tClass2\tOrder2\tFamily2\tGenus2\tSpecies2\tFamilies\tGenera\tSubject1\tSubject2\tARGs", file = CW_onlyARGs_unique_file)

for line in CW_ARGs_unique:
    print(line, file = CW_ARGs_unique_file)
    if line.split("\t")[-1] != "None":
        print(line, file = CW_onlyARGs_unique_file)
        
CW_ARGs_unique_file.close()
CW_onlyARGs_unique_file.close()

In [ ]:
######Looking at the genes found in the shared fragments
sharedARGs = {}

for line in CW_ARGs_unique:
    if line.split("\t")[-1] != "None":
        ARGs = line.split("\t")[24:]
        for item in ARGs:
            ARG = item.split(",")[0]
            if ARG in sharedARGs.keys():
                sharedARGs[ARG] += 1
            else:
                sharedARGs[ARG] = 1

In [ ]:
sharedARGs = sorted(sharedARGs.items(), key=lambda x:x[1], reverse=True)
sharedARGs = dict(sharedARGs)

In [ ]:
sharedARGs

In [ ]:
ARGs_enrichedCows = [
    'DOME_01131|P1-TE_215|gene-1|+|119-1828|complete',
    '1028097032|WP_063853729_1|1|1|mef(En2)|mef(En2)|efflux|2|MACROLIDE|MACROLIDE|macrolide_efflux_MFS_transporter_Mef(En2)',
    '490437755|WP_004308783_1|1|1|lnu(AN2)|lnu(AN2)|nucleotidyltransferase|2|LINCOSAMIDE|LINCOSAMIDE|lincosamide_nucleotidyltransferase_Lnu(AN2)',
    'DLFM02_02210|F11_TET_3|gene-3|+|2650-3888|complete',
    'DLFM11_01245|P17SR4_AZM_210|gene-1|+|88-1308|complete',
    '1988761213|WP_204313008_1|1|1|tet(O_W)|tet(O_W)||2|TETRACYCLINE|TETRACYCLINE|tetracycline_resistance_ribosomal_protection_mosaic_protein_Tet(O_W)',
    'DLFM04_00595|Control-TwinA-Time3_TET_5|gene-3|-|814-2751|complete',
    'DOME_01137|P1-TE_226|gene-2|-|345-695|complete',
    '493486205|WP_006440994_1|1|1|tetA(P)|tetA(P)|efflux|2|TETRACYCLINE|TETRACYCLINE|tetracycline_efflux_MFS_transporter_TetA(P)',
    'DOME_04541|P7-TE_16|gene-1|-|111-2114|complete',
    'DOME_01229|P2-TE_57|gene-1|-|21-269|complete',
    '501051336|WP_012102963_1|1|1|mef(A)|mef(A)|efflux|2|ERYTHROMYCIN|MACROLIDE|macrolide_efflux_MFS_transporter_Mef(A)',
    'DOME_03937|P6-TE_49|gene-2|-|215-1075|complete',
    'DOME_05739|P5-CH_170|gene-1|-|28-813|complete',
    'DLFM07_04648|08X-000_TET_14|gene-3|-|1360-3279|complete',
    'DOME_02841|P4-GE_66|gene-2|-|631-1422|complete'
]

In [ ]:
total = 0
counter = 0

for line in CW_ARGs_unique:
    enriched = False
    if line.split("\t")[-1] != "None":
        total += 1
        ARGs = line.split("\t")[24:]
        for item in ARGs:
            ARG = item.split(",")[0]
            if ARG in ARGs_enrichedCows:
                enriched = True
    if enriched == True:
        counter += 1

In [ ]:
counter

In [ ]:
counter/total

In [ ]:
sharedARGs_file = open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/CW_100cov_sharedARGs.txt", "w")

for key in sharedARGs.keys():
    print(key, file = sharedARGs_file)
    
sharedARGs_file.close()

In [ ]:
sharedARGs_enrichedCows_file = open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/CW_100cov_sharedARGs_EnrichedInCows.txt", "w")

for item in ARGs_enrichedCows:
    print(item, file = sharedARGs_enrichedCows_file)
    
sharedARGs_enrichedCows_file.close()

In [ ]:
###Making a chrod diagram only for ARGs that are enriched in the cow gut
CW_hits_top3CowEnrichedARGonly_top10fam_chord = []

CW_hits_top3CowEnrichedARGonly_top10fam_chord.append("W_family_alt\tC_family_alt\tCount\tW_family\tC_family\tFamilies\tARG")

for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs_nodup_annot_filt_CW_onlyARGs_ShortbredIDs_100cov_uniqueORFs.txt", "r"):
    if line.startswith("Contig"):
        continue
    else:
        line = line.strip().split("\t")
        contig1 = line[0]
        contig2 = line[1]
        
        family1 = line[10]
        family2 = line[17]
        
        if family1 not in CW_hits_top10families:
            family1 = "Other"
            
        if family2 not in CW_hits_top10families:
            family2 = "Other"
        
        if contig1.startswith("W"):
            family_W = "W-" + family1
            family_C = "C-" + family2
        else:
            family_W = "W-" + family2
            family_C = "C-" + family1
            
        families = line[20]
        
        ARGs = line[24:]
        
        for ARG in ARGs:
            ARG = ARG.strip().split(",")
            gene = ARG[0].strip()
            abx_class = ARG[1].strip()
            if gene in ARGs_enrichedCows:
                if abx_class not in ARGs_top3:
                    abx_class = "Other"
                CW_hits_top3CowEnrichedARGonly_top10fam_chord.append("\t".join([family_W, family_C, "1", family_W[2:], family_C[2:], families, abx_class]))

In [ ]:
CW_hits_top3CowEnrichedARGonly_top10fam_chord_file = open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/ChordDiagram/CW_hitsBLASTn_top10families_onlyCowEnrichedARGtop3.txt", "w")

for line in CW_hits_top3CowEnrichedARGonly_top10fam_chord:
    print(line, file = CW_hits_top3CowEnrichedARGonly_top10fam_chord_file)
    
CW_hits_top3CowEnrichedARGonly_top10fam_chord_file.close()

In [ ]:
###Looking at where ARGs are enriched within the shared fragments relative to their prevalence in the MAGs
######Getting the count of all unique ORFs found in the shared fragments
sharedORFcount = 0

unique_hits = []

for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs_nodup_annot_filt_CW_farmerORFs_100cov.txt", "r"):
    if not line.startswith("Contig1"):
        line = line.strip().split("\t")
        ORFs = line[20:]
        for ORF in ORFs:
            if ORF not in unique_hits and ORF != "None":
                unique_hits.append(ORF)
                
sharedORFcount = len(unique_hits)

In [ ]:
######Then, getting the count of all unique ARGs found in the shared fragments
sharedARGcount = 0

unique_hits = []

for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs_nodup_annot_filt_CW_ARGs_100cov.txt", "r"):
    if not line.startswith("Contig1"):
        line = line.strip().split("\t")
        ORFs = line[20:]
        for ORF in ORFs:
            if ORF not in unique_hits and ORF != "None" and ORF.startswith("W"):
                unique_hits.append(ORF)
                
sharedARGcount = len(unique_hits)

In [ ]:
######Then, getting the count of all unique ORFs found in the farmer MAGs
shared_farmer_MAGs = []

for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs_nodup_annot_filt_CW_farmerORFs_100cov.txt", "r"):
    if not line.startswith("Contig1"):
        line = line.strip().split("\t")
        if line[0].startswith("W"):
            MAG = line[12]
        else:
            MAG = line[13]
        
        if MAG not in shared_farmer_MAGs:
            shared_farmer_MAGs.append(MAG)

In [ ]:
allORFcounts = 0

for MAG in shared_farmer_MAGs:
    file = "/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d25_bakta_HQMQ/" + MAG + "/" + MAG + ".faa"
    file = open(file, "r")
    file = file.readlines()
    allORFcounts += len(file)/2

In [ ]:
######Then, getting the count of all unique ARGs found in the farmer MAGs
allARGcounts = 0

MAG_ARG_combos = []

for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d12_shortbred_HQMQ/joint_HQMQ_output.txt", "r"):
    if not line.startswith("MAG"):
        MAG = line.strip().split("\t")[0]
        if MAG in shared_farmer_MAGs:
            if "#" not in line:
                allARGcounts += 1
            else:
                hit = line.strip().split("\t")[1]
                hit = hit.split("_#")[0]
                combo = MAG + "-" + hit
                if combo not in MAG_ARG_combos:
                    MAG_ARG_combos.append(combo)
                    allARGcounts += 1

In [ ]:
print(allORFcounts)
print(allARGcounts)
print(sharedORFcount)
print(sharedARGcount)

In [ ]:
chi2_counts = [[(allORFcounts - allARGcounts - sharedORFcount + sharedARGcount), (allARGcounts - sharedARGcount)],
              [(sharedORFcount - sharedARGcount), sharedARGcount]]

chi2_counts

In [ ]:
chi2_contingency(chi2_counts)

In [8]:
###Looking at microbial families where the MefLnu cassette was found
cow_MAGs = []
cow_families = {}

farmer_MAGs = []
farmer_families = {}

for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs_nodup_annot_filt_CW_onlyARGs_ShortbredIDs_100cov_uniqueORFs.txt"):
    if not line.startswith("Contig1"):
        if "mef(En2)" in line and "Lnu(AN2)" in line:
            line = line.strip().split("\t")
            farmer_MAG = line[4]
            cow_MAG = line[5]
            farmer_family = line[10]
            cow_family = line[17]
            if cow_MAG not in cow_MAGs:
                cow_MAGs.append(cow_MAG)
                
                if cow_family not in cow_families.keys():
                    cow_families[cow_family] = 1
                else:
                    cow_families[cow_family] += 1
                    
            if farmer_MAG not in farmer_MAGs:
                farmer_MAGs.append(farmer_MAG)
                
                if farmer_family not in farmer_families.keys():
                    farmer_families[farmer_family] = 1
                else:
                    farmer_families[farmer_family] += 1

In [13]:
cow_families

{'f__Paludibacteraceae': 1,
 'f__Lachnospiraceae': 8,
 'f__P3': 2,
 'f__Bacteroidaceae': 3}

In [ ]:
###Doing all of the above but for office worker and cows now
###First gettin all of the BLAST hits between office workers and cows
CD_hits = []

for line in open("/Users/bejanmahmud/Library/CloudStorage/Box-Box/Bejan's files/Projects/DOME/DOME/Analysis/Metagenome/220221_DOME_fc_all/d22_BLASTn_HQMQ/joint_BLASTn_allHQMQMAGs_nodup_annot_filt.txt", "r"):
    if line.startswith("Contig1"):
        continue
    else:
        line = line.strip()
        if "C-D" in line:
            CD_hits.append(line)

In [ ]:
######Now (as above), parsing through the BLASTn to get the encoded ARGs
CD_hits_ARG = []

for line in CD_hits:
    contig1 = line.split("\t")[0]
    contig2 = line.split("\t")[1]
    
    start1 = int(line.split("\t")[6])
    stop1 = int(line.split("\t")[7])
    
    if start1 > stop1:
        temp = start1
        start1 = stop1
        stop1 = temp
        
    start2 = int(line.split("\t")[8])
    stop2 = int(line.split("\t")[9])
    
    if start2 > stop2:
        temp = start2
        start2 = stop2
        stop2 = temp
    
    ARGs = ""
    
    if contig1 not in contig_ARGcoordinates.keys() and contig2 not in contig_ARGcoordinates.keys():
        CD_hits_ARG.append(line + "\tNone")
    
    else:
        
        if contig1 in contig_ARGcoordinates.keys():
            for i in range(len(contig_ARGcoordinates[contig1])):
                start = int(contig_ARGcoordinates[contig1][i].split("\t")[0])
                stop = int(contig_ARGcoordinates[contig1][i].split("\t")[1])
                cov = 0

                if start > stop:
                    temp = start
                    start = stop
                    stop = temp

                if start > start1 and start < stop1 and stop > start1 and stop < stop1:
                    cov = 100
                elif start > start1 and start < stop1 and stop > start1 and stop > stop1:
                    cov = (stop1 - start)/(stop - start)
                    cov = round(cov * 100)
                elif start < start1 and start < stop1 and stop > start1 and stop < stop1:
                    cov = (stop - start1)/(stop - start)
                    cov = round(cov * 100)
                elif start < start1 and start < stop1 and stop > start1 and stop > stop1:
                    cov = (stop1 - start1)/(stop - start)
                    cov = round(cov * 100)
                else:
                    cov = 0
                    
                if cov == 100:
                    if ARGs == "":
                        ARGs = contig1 + "," + ARG_ORFs[contig_ARGORFID[contig1][i]] + "," + contig_abxClasses[contig1][i] + "," + contig_ARGannot[contig1][i] + "," + str(cov)
                    else:
                        ARGs = ARGs + "\t" + contig1 + "," + ARG_ORFs[contig_ARGORFID[contig1][i]] + "," + contig_abxClasses[contig1][i] + "," + contig_ARGannot[contig1][i] + "," + str(cov)
                        
        if contig2 in contig_ARGcoordinates.keys():
            for i in range(len(contig_ARGcoordinates[contig2])):
                start = int(contig_ARGcoordinates[contig2][i].split("\t")[0])
                stop = int(contig_ARGcoordinates[contig2][i].split("\t")[1])
                cov = 0

                if start > stop:
                    temp = start
                    start = stop
                    stop = temp

                if start > start2 and start < stop2 and stop > start2 and stop < stop2:
                    cov = 100
                elif start > start2 and start < stop2 and stop > start2 and stop > stop2:
                    cov = (stop2 - start)/(stop - start)
                    cov = round(cov * 100)
                elif start < start2 and start < stop2 and stop > start2 and stop < stop2:
                    cov = (stop - start2)/(stop - start)
                    cov = round(cov * 100)
                elif start < start2 and start < stop2 and stop > start2 and stop > stop2:
                    cov = (stop2 - start2)/(stop - start)
                    cov = round(cov * 100)
                else:
                    cov = 0
                    
                if cov == 100:
                    if ARGs == "":
                        ARGs = contig2 + "," + ARG_ORFs[contig_ARGORFID[contig2][i]] + "," + contig_abxClasses[contig2][i] + "," + contig_ARGannot[contig2][i] + "," + str(cov)
                    else:
                        ARGs = ARGs + "\t" + contig2 + "," + ARG_ORFs[contig_ARGORFID[contig2][i]] + "," + contig_abxClasses[contig2][i] + "," + contig_ARGannot[contig2][i] + "," + str(cov)
                        
        if ARGs == "":
            CD_hits_ARG.append(line + "\tNone")
        else:
            CD_hits_ARG.append(line + "\t" + ARGs)

In [ ]:
###Now going thorugh the entries to keep only unique ARG-matched ORFs for each BLAST hit
CD_ARGs_unique = []

for line in CD_hits_ARG:
    line = line.strip().split("\t")

    contig1 = line[0]
    contig2 = line[1]

    pident = line[2]
    length = line[3]

    MAG1 = line[12]
    MAG2 = line[13]

    taxa1 = MAG_taxa[MAG1]
    taxa1 = taxa1.split(";")
    domain1 = taxa1[0]
    phylum1 = taxa1[1]
    class1 = taxa1[2]
    order1 = taxa1[3]
    family1 = taxa1[4]
    genus1 = taxa1[5]
    species1 = taxa1[6]

    taxa2 = MAG_taxa[MAG2]
    taxa2 = taxa2.split(";")
    domain2 = taxa2[0]
    phylum2 = taxa2[1]
    class2 = taxa2[2]
    order2 = taxa2[3]
    family2 = taxa2[4]
    genus2 = taxa2[5]
    species2 = taxa2[6]

    if family1 == family2:
        families = "Same family"
    else:
        families = "Different families"

    if genus1 == genus2:
        genera = "Same genera"
    else:
        genera = "Different genera"

    subject1 = line[16]
    subject2 = line[17]

    ARGs = line[20:]

    if line[-1] == "None":
        CD_ARGs_unique.append("\t".join([contig1, contig2, pident, length, MAG1, MAG2, domain1, phylum1, class1, order1, family1, genus1, species1, domain2, phylum2, class2, order2, family2, genus2, species2, families, genera, subject1, subject2, line[-1]]))
    else:
        unique_ORFs = []
        unique_entries = []

        for item in ARGs:
            item = item.split(",")
            ARG = item[1]
            ARGclass = item[2]
            ORF = item[3]
            if ORF not in unique_ORFs:
                unique_entries.append(f"{ARG}, {ARGclass}, {ORF}")
                unique_ORFs.append(ORF)

        unique_entries = "\t".join(unique_entries)
        CD_ARGs_unique.append("\t".join([contig1, contig2, pident, length, MAG1, MAG2, domain1, phylum1, class1, order1, family1, genus1, species1, domain2, phylum2, class2, order2, family2, genus2, species2, families, genera, subject1, subject2, unique_entries]))

In [ ]:
######Looking at the genes found in the shared fragments
sharedARGs_CD = {}

for line in CD_ARGs_unique:
    if line.split("\t")[-1] != "None":
        ARGs = line.split("\t")[24:]
        for item in ARGs:
            ARG = item.split(",")[0]
            if ARG in sharedARGs_CD.keys():
                sharedARGs_CD[ARG] += 1
            else:
                sharedARGs_CD[ARG] = 1

In [ ]:
sharedARGs_CD = sorted(sharedARGs_CD.items(), key=lambda x:x[1], reverse=True)
sharedARGs_CD = dict(sharedARGs_CD)

In [ ]:
sharedARGs_CD

In [ ]:
ARGs_enrichedCows = ARGs_enrichedCows + ["DOME_00805|P6-CH_34|gene-2|-|603-1037|complete", "DOME_05731|P5-CH_149|gene-1|-|298-708|complete"]

In [ ]:
ARGs_enrichedCows

In [ ]:
counter = 0
total = 0

for key, value in sharedARGs_CD.items():
    total += value
    if key in ARGs_enrichedCows:
        counter += value
        
counter/total

In [ ]:
total